
Fully Convolutional Network (FCN) for Semantic Segmentation
============================================================
This notebook implements FCN variants with different upsampling strategies
and compares their performance on a semantic segmentation task.


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.datasets import VOCSegmentation
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [4]:
# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
